In [1]:
import numpy as np
# from tqdm import tqdm
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import Legend
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead

from scipy.optimize import root
#from scipy.optimize import fsolve, root

output_notebook(hide_banner=True)

# Introduction

**The aim of this session** is to learn the distinction between *explicit* and *implicit* methods, to learn how to implement some of them as well as to find ways to test them. We will also introduce the notion of order of convergence and explain how to observe the order of convergence of a method in practical situations.

## Context

For $t_0,t_f\in\mathbb{R}$, $t_f>t_0$, consider the Cauchy problem
\begin{equation}
\begin{array}{|rcl}
y'(t)&=&f(t,y(t)),\quad t\geq t_0\\[3pt]
y(t_0)&=&y_0
\end{array}
\end{equation}
which we will refer to as Problem (1), where $f$ satisfies the assumptions of the Cauchy-Lipschitz theorem. We assume that the unique maximal solution to this problem is defined for all $t\in [t_0;t_f]$. In order to approximate the solution of Problem (1), let us consider a few simple numerical schemes. To proceed, first for $h>0$ small, we define the discrete times $t_n:=t_0+nh$ with $n=0,1,2,\dots,N$. Here $N$ stands for the maximal integer such that $t_{N}\le t_f$. The parameter $h$ is called the discretization stepsize. Our goal is to construct a sequence of values $u_n$ to approximate the exact solution at time $t_n$, that is to approximate $y(t_n)$. We start by setting $u_0=y_0$. Then we construct the sequence $u_n$ iteratively. Below we focus our attention on one-steps methods for which $u_{n+1}$ depends only on $u_n$ (and not on $u_{n-1}$, $u_{n-2}$, $\dots$). Here are some of them:

a) Forward Euler method
$$
u_{n+1}=u_n+hf(t_n,u_n).
$$
b) Backward Euler method
$$
u_{n+1}=u_n+hf(t_{n+1},u_{n+1}).
$$
c) Trapezoidal (or Cranck-Nicolson) method
$$
u_{n+1}=u_n+\cfrac{h}{2}\,(f(t_{n},u_{n})+f(t_{n+1},u_{n+1})).
$$
d) Heun method
$$
u_{n+1}=u_n+\cfrac{h}{2}\,(f(t_{n},u_{n})+f(t_{n+1},u_{n}+hf(t_{n},u_{n}))).
$$
e) Second order Taylor-series method (TS2)
$$
u_{n+1} = u_n +hf(t_n,u_n)+\frac{h^2}{2}g(t_n,u_n)
$$
where $g:(t,y)\mapsto \partial_t f(t,y) + \partial_y f(t,y) f(t,y)$ so that we have :
$$
\begin{aligned}
\frac{\text{d}f}{\text{d}t}\left(t,y(t)\right) &= \partial_t f\left(t,y(t)\right) + \partial_y f\left(t,y(t)\right) y^\prime(t)\\
&=\partial_t f\left(t,y(t)\right) + \partial_y f\left(t,y(t)\right) f\left(t,y(t)\right)\\
&= g(t,y(t))
\end{aligned}
$$

**Q1. What is the definition of an explicit (resp. implicit method)? Which of these methods are explicit (resp. Implicit)?**

Explicit methods : If we have $u_n$, then we can compute $u_{n+1}$. Examples are a) Forward Euler Method, d) Heun Method, and e) Second-order Taylor-series Method. 


Implicit methods : The terms $u_n$ appears on both sides of the equation, meaning that we cannot compute it directly. We need to solve the equation instead. Examples are b) Backward Euler Method, and c) Trapezoidal Method. 

**Simplifiying assumption**: In the present notebook only scalar problems are considered to keep relatively simple implementations. We also assume that step size $h$ is always such that $t_f-t_0 = N h$ with $N \in \mathbb{N}$. One should always remember that if this condition does not hold then the methods presented here will not reach the correct final time $t_f$.

## Model Problem

As stated in the introduction, it is crucial to have ways to test your algorithms, whether it is because you are designing a new scheme that requires validation or because you might want to debug your implementation of a classic and well known method. An approach to do so is to consider a model problem for which you can derive an exact solution.

In this session we will consider the model problem 
$$
f_1(t,y) = (1-2t)y,\quad y(0) = 1.
$$
Using the method of variable separation (see TD1) one can find that the solution to the corresponding initial value problem is
$$
y_{sol}^1 : t \mapsto \exp \left(\frac{1}{4} - \left(t-\frac{1}{2}\right)^2\right)
$$

In [2]:
def f1(t,y):
    return (1 - 2*t) * y

In [3]:
def y1_exa(t):
    return np.exp(1/4 - (1/2 - t)**2)

**Q2. Derive the function $g_1$ correspondition to this model problem and that will be needed for the implementation of the TS2 method (see definition in the introduction). Implement it.**

*Please write your answer here*

In [5]:
def g1(t,y): 
    return -2*y + ((1 - 2*t)**2)*y 

# Part I: Explicit methods

In this part we will implement the explict methods, that is we will implement the forward Euler method, the Heun method, and the second order Taylor series method.

### Forward Euler method
We recall the iterative formula for the forward Euler method:
$$
u_{n+1} = u_n + hf(t_n,u_n)
$$
where $t_n = t_0 + nh$ and $n \in \{0,...,N\}$ with $N = \frac{t_f-t_0}{h}+1$ (because of our simplifiying assumption $N\in\mathbb{N}$).

**Q3. Implement the forward Euler method. The method should take as argument the initial time $t_0$, final time $t_f$, step size $h$, a starting point $y_0 = y(t_0)$ and a function $f$. It should  return an array of size $nt$ where $nt = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor+1$.**

In [19]:
def Euler(f, t0, tf, h, y0):
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    u  = np.zeros((nt,))         #array of solutions to be returned by the function

    t  = [t0 + i*h for i in range(nt)] 
    for i in range(nt): 
        if (i == 0): 
            u[i] = f(t0, y0) 
        else: 
            u[i] = u[i-1] + h*f(t[i-1], u[i-1])
        
    return u

### Taylor Series (TS) Method

For the TS method we want the iterative formula to be :
$$
u_{n+1} = u_n + hf(t_n,u_n)+ \frac{h^2}{2} g(t_n,u_n)
$$
where $g:(t,y)\mapsto \partial_t f(t,y) + \partial_x f(t,y) f(t,y)$ so that we have :
$$
\begin{aligned}
\frac{\text{d}f}{\text{d}t}\left(t,y(t)\right) &= \partial_t f\left(t,y(t)\right) + \partial_x f\left(t,y(t)\right) y^\prime(t)\\
&=\partial_t f\left(t,y(t)\right) + \partial_x f\left(t,y(t)\right) f\left(t,y(t)\right)\\
&= g(t,y=y(t))
\end{aligned}
$$

In the case of our model problem $f_1$ we have for instance the function $g_1$ defined for $(t,y) \in \mathbb{R}_+\times \mathbb{R}$ by
$$
g_1(t,y) = \partial_t f_1(t,y) + \partial_x f_1(t,y) f_1(t,y) = -2 y + (1-2t)(1-2t)y = \left((1-2t)^2-2\right) y
$$

**Q4. Implement the second order Taylor series (TS2) method. The method should take as argument the initial time $t_0$, final time $t_f$, step size $h$, a starting point $y_0 = y(t_0)$ and the functions $f$ and $g$. It should  return an array of size $nt$ where $nt = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor+1$.**

In [20]:
def g1(t,y):
    return ((1-2*t)**2-2)*y

In [21]:
def TS2(f, g, t0, tf, h, u0):
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    u  = np.zeros((nt,))
    
    t  = [t0 + i*h for i in range(nt)] 
    for i in range(nt): 
        if (i == 0): 
            u[i] = f(t0, y0) 
        else: 
            u[i] = u[i-1] + h*f(t[i-1], u[i-1]) + (h**2/2)*g(t[i-1], u[i-1]) 

    return u

### Heun method

We recall the Heun method
$$
u_{n+1}=u_n+\cfrac{h}{2}\,(f(t_{n},u_{n})+f(t_{n+1},u_{n}+hf(t_{n},u_{n}))).
$$

**Q5. Implement the Heun method. The method should take as argument the initial time $t_0$, final time $t_f$, step size $h$, a starting point $y_0 = y(t_0)$ and the function $f$. It should  return an array of size $nt$ where $nt = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor+1$.**

In [22]:
def Heun(f, t0, tf, h, y0):
    
    nt = int(round((tf-t0)/h))+1 #number of columns of the output array
    u  = np.zeros((nt,))         #array of solutions to be returned by the function
    
    t  = [t0 + i*h for i in range(nt)] 
    for i in range(nt): 
        if (i == 0): 
            u[i] = f(t0, y0) 
        else: 
            u[i] = u[i-1] + (h/2)*(f(t[i-1], u[i-1]) + f(t[i], u[i-1] + h*f(t[i-1], u[i-1]))) 

    return u

### Comparison

**Q6. As a first step to verify that your implementation is correct, compare these methods on a graph. Also check the global error term on the second graph. You do not need to write more code. If the code you wrote for Q1 is correct, the code below should run and produce what is expected. Please write a small comment telling us whether you are satisfied with what you observe or not.**

In [25]:
t0 = 0.  #initial time
tf = 4.0 #final time
h  = 0.2 #time step
y0 = 1.  #starting point

u_Elr  = Euler(f1, t0, tf, h, y0)
u_TS2  = TS2(f1, g1, t0, tf, h, y0)
u_Heun = Heun(f1, t0, tf, h, y0)

nt = np.size(u_Elr)
t  = t0+1.*h*np.arange(nt)

N = 100 # number of points for the exact solution
texa = np.linspace(t0,tf,N)
yexa1 = y1_exa(texa)

fig1 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='x(t)')
fig1.line(texa,yexa1,legend_label = 'Exact solution')
fig1.circle(t,u_Elr, color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig1.circle(t,u_TS2,fill_color = 'white', legend_label = 'TS method approximation', size = 8)
fig1.x(t,u_Heun,color = 'red', legend_label = 'Heun method approximation', size = 8)

yexa2 = y1_exa(t)
fig2 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='GE')
fig2.circle(t,abs(yexa2-u_Elr), color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig2.circle(t,abs(yexa2-u_TS2),fill_color = 'white', legend_label = 'TS method approximation', size = 8)
fig2.x(t,abs(yexa2-u_Heun),color = 'red', legend_label = 'Heun method approximation', size = 8)

show(column(fig1,fig2))

*Please write your answer here.*

### Convergence study

Comparing methods visually on an example is very usefull to detect the most obvious bugs and mistakes in the implementation. It does not allow however to check the theoretical order of the method. In this question we adress this specific aspect.

We recall that a discrete equivalent of the $L^2$-norm $||u||_{L^2\left([t_0,t_f]\right)} = \sqrt{\int^{t_f}_{t_0}u^2(t)\text{d}t}$ of a continuous function $u\in \mathcal{C}^0\left([t_0,t_f]\right)$ can be defined by:
$$
\sqrt{\frac{1}{N}\sum^N_{n=0}u^2(t_n)}
$$
with $t_n = t_0 + nh$ where $h$ is the step size, $n\in\{0,...,N\}$ and $N = \left\lfloor\frac{t_f-t_0}{h}\right\rfloor$. It is called the $l^2$-norm.
To assess the accuracy of a numerical approximation $\left(u_n\right)_{n\in\{0,...,N\}}$, it can be useful to evaluate the following quantity: (error in discrete $l^2$-norm)
$$
e_{l^2\text{-}norm} = \sqrt{\frac{1}{N}\sum^N_{n=0}\left(u(t_n)-u_n\right)^2}
$$
We shall want to show that $e_{l^2-norm} = \mathcal{O}\left(h^p\right)$ for an approximation of order $p$. This is equivalent to $|e_{l^2\text{-}norm}|\leq C h^p$ with $C>0$.
For convergence studies, we usually use log scales to show that this upper bound holds:
$$
\ln |e_{l^2\text{-}norm}|\leq p \ln (h) + \ln (C)
$$
Indeed, under this formalism, satisfying this inequality is graphically equivalent to remaining below a straight line.

**Q7. To complete the process of checking the validity of your methods, study the order of convergence in terms of $l^2$-norm for the implemented methods using log-scale graphs.
You do not need to write more code. If the code you wrote for Q1 is correct, the code below should run and produce what is expected. Please write a small comment telling us whether you are satisfied with what you observe or not.**

In [26]:
t0 = 0.  #initial time
tf = 4.0 #final time 
y0 = 1.  #initial condition

nh         = 10                        #number of possible values for h
n_mthd     = 3                         #number of methods to be tested
h_values   = 10**np.linspace(-4,-1,nh) #values of h
err_values = np.zeros((n_mthd,nh))     #error values for each method and each h
ith=0                                  #index corresponding to the current h in h_value

for h in h_values:
    u_Elr  = Euler(f1, t0, tf, h, y0)
    u_TS2  = TS2(f1, g1, t0, tf, h, y0)
    u_Heun = Heun(f1, t0, tf, h, y0)
    
    nt = np.size(u_Elr)
    t = t0+1.*h*np.arange(nt)
    yexa = y1_exa(t)
    err_values[0,ith] = np.linalg.norm(yexa-u_Elr)/np.sqrt(nt)
    err_values[1,ith] = np.linalg.norm(yexa-u_TS2)/np.sqrt(nt)
    err_values[2,ith] = np.linalg.norm(yexa-u_Heun)/np.sqrt(nt)
    ith+=1

fig3 = figure( x_axis_type="log", y_axis_type="log", width=980, height=400, x_axis_label='h', y_axis_label='GE')
fig3.circle(h_values,err_values[0,:], color = 'navy', legend_label = 'Euler method approximation', size = 8)
fig3.circle(h_values,err_values[1,:],fill_color = 'white', legend_label = 'TS2 method approximation', size = 8)
fig3.x(h_values,err_values[2,:],color = 'red', legend_label = 'Heun method approximation', size = 8)

fig3.line(h_values,h_values,legend_label = 'order 1 slope')
fig3.line(h_values,h_values**2,color ='red' ,legend_label = 'order 2 slope')
fig3.legend.location = "bottom_right"
show(fig3)

*Please write your answer here.*

# Part II: Implicit methods

In this part we will implement the implicit methods, that is we will implement the backward Euler method and the trapezoidal rule.

One the main challenge in implementing implicit methods is that in general they require to solve a non linear equation. This step in itself can be the source of multiple mistakes or bugs in our implementation. Consequently it is always a good habit to start implementing such method on a particular simple case first before considering the general implementation.

That is how we will proceed in this part. First we will implement the backward Euler method spefically for the model problem $f_1$, which is linear, and subsequently we will consider the general case.

### The specific case of the model problem 

**Q8. Derive the formulas for the backward Euler method and Trapezoidal rule specifically in the case of the model problem.**

*Please write your answer here.*

For the Backward Euler Method, the formula is 
\begin{equation} 
    U_{n+1} = \frac{1}{1 - h \left( 1 - 2 t_{n+1} \right)} U_{n} 
\end{equation}

For the Trapezoidal Method, the formula is 
\begin{equation} 
    U_{n+1} = \frac{1 + \frac{h}{2} \left( 1 - 2 t_{n} \right)}{1 - \frac{h}{2} \left( 1 - 2 t_{n+1} \right)} U_{n} 
\end{equation} 

**Q9. Implement these problem-specific versions of the Backward Euler method and Trapezoidal rule.**

In [32]:
def backEuler_1(t0, tf, h, y0): #Method specific to the IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    u  = np.zeros((nt,))
    
    t  = [t0 + i*h for i in range(nt)] 
    for i in range(nt): 
        if (i == 0): 
            u[i] = f1(t0, y0) 
        else: 
            u[i] = (1 / (1 - h*(1 - 2*t[i]))) * u[i-1] 
        
    return u

In [33]:
def TR_1(t0, tf, h, y0): #Method specific to the IVP (1)
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    u  = np.zeros((nt,))
    
    t  = [t0 + i*h for i in range(nt)] 
    for i in range(nt): 
        if (i == 0): 
            u[i] = f1(t0, y0) 
        else: 
            u[i] = ((1 + (h/2)*(1 - 2*t[i-1])) / (1 - (h/2)*(1 - 2*t[i]))) * u[i-1]
    
    return u

**Q10. In the manner of Q6. and Q7. verify that your methods are correctly implemented and yield the expected order of convergence**

In [35]:
t0 = 0.  #initial time
tf = 4.0 #final time
h  = 0.2 #time step
y0 = 1.  #starting point

u_BEM = backEuler_1(t0, tf, h, y0) 
u_TR  = TR_1(t0, tf, h, y0) 

nt = np.size(u_BEM) 
t  = t0+1.*h*np.arange(nt)

N = 100 # number of points for the exact solution
texa = np.linspace(t0,tf,N)
yexa1 = y1_exa(texa)

fig1 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='x(t)')
fig1.line(texa,yexa1,legend_label = 'Exact solution')
fig1.circle(t,u_BEM, color = 'navy', legend_label = 'Backward Euler method approximation', size = 8)
fig1.circle(t,u_TR,fill_color = 'white', legend_label = 'Trapezoidal method approximation', size = 8)

yexa2 = y1_exa(t)
fig2 = figure(x_range=(t0, tf), width=980, height=400, x_axis_label='t', y_axis_label='GE')
fig2.circle(t,abs(yexa2-u_BEM), color = 'navy', legend_label = 'Backward Euler method approximation', size = 8)
fig2.circle(t,abs(yexa2-u_TR),fill_color = 'white', legend_label = 'Trapezoidal method approximation', size = 8)

show(column(fig1,fig2))

In [37]:
t0 = 0.  #initial time
tf = 4.0 #final time 
y0 = 1.  #initial condition

nh         = 10                        #number of possible values for h
n_mthd     = 3                         #number of methods to be tested
h_values   = 10**np.linspace(-4,-1,nh) #values of h
err_values = np.zeros((n_mthd,nh))     #error values for each method and each h
ith=0                                  #index corresponding to the current h in h_value

for h in h_values:
    u_BEM = backEuler_1(t0, tf, h, y0) 
    u_TR  = TR_1(t0, tf, h, y0) 

    
    nt = np.size(u_BEM)
    t = t0+1.*h*np.arange(nt)
    yexa = y1_exa(t)
    err_values[0,ith] = np.linalg.norm(yexa-u_BEM)/np.sqrt(nt)
    err_values[1,ith] = np.linalg.norm(yexa-u_TR)/np.sqrt(nt)
    ith+=1

fig3 = figure( x_axis_type="log", y_axis_type="log", width=980, height=400, x_axis_label='h', y_axis_label='GE')
fig3.circle(h_values,err_values[0,:], color = 'navy', legend_label = 'Backward Euler method approximation', size = 8)
fig3.circle(h_values,err_values[1,:],fill_color = 'white', legend_label = 'Trapezoidal method approximation', size = 8)

fig3.line(h_values,h_values,legend_label = 'order 1 slope')
fig3.line(h_values,h_values**2,color ='red' ,legend_label = 'order 2 slope')
fig3.legend.location = "bottom_right"

show(fig3)

### General case

**Q11. Using the fonction root of scipy.optimize, implement the backward Euler method and the Trapezoidal Rule method for a generic IVP associated to a function $f$**

In [39]:
def backEuler(f, t0, tf, h, y0):
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    u  = np.zeros((nt,))
    
    #########################
    #To be completed
    #########################

    return u

In [44]:
def TR(f, t0, tf, h, y0):
    
    nt = int(round((tf-t0)/h))+1 #number of column of the output array
    u  = np.zeros((nt,))
    
    #########################
    #To be completed
    #########################
    
    return u

**Q12. Check that in the case of our model problem, these general methods yield results  identical to that of previous implementations. Also test the case of a nonlinear ODE.**

In [24]:
#########################
#To be completed
#########################